# Amadeus API to get Hotel Offers

- [example-building-hotel-booking-engine](https://developers.amadeus.com/blog/build-hotel-booking-engine-amadeus-api)
- [hotel-list-api](https://developers.amadeus.com/self-service/category/hotels/api-doc/hotel-list/api-reference)
- [hotel-search-api](https://developers.amadeus.com/self-service/category/hotels/api-doc/hotel-search/api-reference)
- [hotel-ratings-api](https://developers.amadeus.com/self-service/category/hotels/api-doc/hotel-ratings/api-reference)

In [39]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import json
import os
import time

# Load the CSV file containing city data
cities_df = pd.read_csv('cities_data.csv')

# Function to get access token
def get_access_token(client_id, client_secret):
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Failed to get access token: {response.status_code}, {response.text}")

# Retry decorator
def retry_on_failure(retries=5, backoff_factor=1, status_code=429):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(retries):
                result = func(*args, **kwargs)
                if isinstance(result, tuple) and result[1] == status_code:
                    print(f"Rate limit exceeded. Retrying in {backoff_factor * (2 ** attempt)} seconds...")
                    time.sleep(backoff_factor * (2 ** attempt))
                else:
                    return result
            return func(*args, **kwargs)  # Final attempt
        return wrapper
    return decorator

# Function to get hotel IDs by geocode
@retry_on_failure(retries=10, backoff_factor=2, status_code=429)
def get_hotel_ids_by_geocode(access_token, latitude, longitude):
    url = f"https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-geocode"
    params = {
        "latitude": latitude,
        "longitude": longitude
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers, params=params)
    hotel_ids = []
    if response.status_code == 200:
        data = response.json()
        for hotel in data['data']:
            hotel_ids.append(hotel['hotelId'])
        return hotel_ids, 200
    else:
        print(f"Failed to fetch hotel IDs: {response.status_code}, {response.text}")
        return [], response.status_code

# Function to get hotel offers
@retry_on_failure(retries=10, backoff_factor=2, status_code=429)
def get_hotel_offers(access_token, hotel_ids, check_in_date, adults=1, room_quantity=1, currency="EUR",  city="",country_code="", country=""):
    url = f"https://test.api.amadeus.com/v3/shopping/hotel-offers"
    params = {
        "hotelIds": ",".join(hotel_ids),
        "checkInDate": check_in_date,
        "adults": adults,
        "roomQuantity": room_quantity,
        "currency": currency
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers, params=params)
    hotels = []
    if response.status_code == 200:
        data = response.json()
        for offer in data['data']:
            hotel = offer['hotel']
            for offer_detail in offer['offers']:
                hotel_info = {
                    'hotel_id': hotel['hotelId'],
                    'hotel_name': hotel['name'],
                    'city': city,
                    'country_code': country_code,
                    'country': country,
                    'latitude': hotel['latitude'],
                    'longitude': hotel['longitude'],
                    'check_in_date': offer_detail['checkInDate'],
                    'check_out_date': offer_detail['checkOutDate'],
                    'price_per_night': offer_detail['price']['total'],
                    'currency': offer_detail['price']['currency'],
                    'room_type': offer_detail['room']['type'],
                    'room_description': offer_detail['room']['description']['text']
                }
            hotels.append(hotel_info)
        return hotels, 200
    else:
        print(f"Failed to fetch hotel offers: {response.status_code}, {response.text}")
        return [], response.status_code

# Function to get hotel ratings
@retry_on_failure(retries=10, backoff_factor=2, status_code=429)
def get_hotel_ratings(access_token, hotel_ids):
    url = f"https://test.api.amadeus.com/v2/e-reputation/hotel-sentiments"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    ratings = {}
    for i in range(0, len(hotel_ids), 3):
        batch_ids = hotel_ids[i:i+3]
        params = {
            "hotelIds": ",".join(batch_ids)
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'data' not in data:
                continue
            for sentiment in data['data']:
                ratings[sentiment['hotelId']] = sentiment['overallRating']
        else:
            print(f"Failed to fetch hotel ratings: {response.status_code}, {response.text}")
    return ratings, 200

# Function to save state
def save_state(state, filename="hotel_state.json"):
    with open(filename, 'w') as f:
        json.dump(state, f)

# Function to load state
def load_state(filename="hotel_state.json"):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except:
        return None

# Function to save hotel data
def save_hotel_data(hotel_data, filename="hotel_data.csv"):
    hotel_df = pd.DataFrame(hotel_data)
    hotel_df.to_csv(filename, index=False)

# Function to load hotel data
def load_hotel_data(filename="hotel_data.csv"):
    try:
        hotel_df = pd.read_csv(filename)
        return hotel_df.to_dict('records')
    except:
        return []

In [40]:
# API Key and Secret for Amadeus
client_id = "IGnGRFWIONOz3O5GcB97FMh9GFgOYCGG"
client_secret = "f0ao9LtaW4UZiaU8"

# Get access token
access_token = get_access_token(client_id, client_secret)

# Load previous state if it exists
state = load_state()
if state:
    last_city = state['last_city']
    last_country_code = state['last_country_code']
    last_date_str = state['last_date']
    hotel_data = load_hotel_data()
else:
    last_city = None
    last_country_code = None
    last_date_str = None
    hotel_data = []

# If a date was saved, start from that date
if last_date_str:
    start_date = datetime.strptime(last_date_str, '%Y-%m-%d')
else:
    start_date = datetime(2024, 7, 3)

end_date = datetime(2024, 7, 31)
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]

# Iterate over cities and dates to fetch data
for _, row in cities_df.iterrows():
    city = row['name']
    country = row['country']
    country_code = row['country_code']
    latitude = row['latitude']
    longitude = row['longitude']
    if last_city and last_country_code and (city != last_city or country_code != last_country_code):
        continue

    for date in date_generated:
        if last_date_str and date.strftime('%Y-%m-%d') <= last_date_str:
            continue

        date_str = date.strftime('%Y-%m-%d')
        hotel_ids, _ = get_hotel_ids_by_geocode(access_token, latitude, longitude)
        if hotel_ids:
            hotels, _ = get_hotel_offers(access_token, hotel_ids, date_str, city=city, country_code=country_code, country=country)
            if hotels:
                # Add ratings to hotel data
                hotel_ids_for_rating = [hotel['hotel_id'] for hotel in hotels]
                ratings, _ = get_hotel_ratings(access_token, hotel_ids_for_rating)
                for hotel in hotels:
                    hotel['rating'] = ratings.get(hotel['hotel_id'], None)
                hotel_data.extend(hotels)

        # Save state and hotel data after each request
        state = {
            'last_city': city,
            'last_country_code': country_code,
            'last_date': date_str,
            'hotel_data': hotel_data
        }
        save_state(state)
        save_hotel_data(hotel_data)

# Convert hotel data to DataFrame
hotel_df = pd.DataFrame(hotel_data)

# Save data to CSV file
hotel_df.to_csv('hotel_data.csv', index=False)

# Delete saved state after completing the script
if os.path.exists("hotel_state.json"):
    os.remove("hotel_state.json")

print("Data saved to 'hotel_data.csv'")

Failed to fetch hotel offers: 400, {"errors":[{"code":1351,"title":"VERIFY CHAIN/REP CODE","status":400,"detail":"Provider Error - VERIFY CHAIN/REP CODE","source":{"parameter":"hotelIds=FGDXB211"}}]}
Failed to fetch hotel offers: 400, {"errors":[{"code":1351,"title":"VERIFY CHAIN/REP CODE","status":400,"detail":"Provider Error - VERIFY CHAIN/REP CODE","source":{"parameter":"hotelIds=FGDXB211"}}]}
Failed to fetch hotel offers: 400, {"errors":[{"code":1351,"title":"VERIFY CHAIN/REP CODE","status":400,"detail":"Provider Error - VERIFY CHAIN/REP CODE","source":{"parameter":"hotelIds=FGDXB211"}}]}
Failed to fetch hotel offers: 400, {"errors":[{"code":1351,"title":"VERIFY CHAIN/REP CODE","status":400,"detail":"Provider Error - VERIFY CHAIN/REP CODE","source":{"parameter":"hotelIds=FGDXB211"}}]}
Failed to fetch hotel offers: 400, {"errors":[{"code":1351,"title":"VERIFY CHAIN/REP CODE","status":400,"detail":"Provider Error - VERIFY CHAIN/REP CODE","source":{"parameter":"hotelIds=FGDXB211"}}]}


KeyboardInterrupt: 